# <span style='color:lime;'>What are Kernel Functions</span>
### Kernel Functions gives ability LLM to interact with native code which is written in the programming language that supported by Semantic Kernel SDK. Which helps to perform some complex tasks that can easily done using our own code instead of relying everything on LLM. Which is very critical in any LLM applications. This Semantic Kernel SDK will intelligently invoke the functions required. These are ideally coupled with Plugins. 


# <span style='color:lime;'>How to create Kernel Functions</span>
### What are different ways to create functions
### How to use them
### Calling Functions
### Automatic function calling
### Nested Function calling


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: DotNetEnv, 2.2.0"
#r "nuget: Microsoft.Extensions.Logging.Console, *"
#r "nuget: Microsoft.Extensions.DependencyInjection,*"

Installed Packages DotNetEnv, 2.2.0 Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.SemanticKernel, 1.7.1

In [2]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.Diagnostics;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.ChatCompletion;

Env.Load("../");
var endpoint = Environment.GetEnvironmentVariable("AOAIEndPoint");
var apikey = Environment.GetEnvironmentVariable("AOAIApiKey");
var deployementName = Environment.GetEnvironmentVariable("deployementName");


In [9]:

var skBuilder = Kernel.CreateBuilder();

//for detailed verbose
skBuilder.Services.AddLogging(configure => configure.AddConsole().SetMinimumLevel(LogLevel.Trace));

var kernel = skBuilder
.AddAzureOpenAIChatCompletion(deployementName,endpoint,apikey)
.Build();

In [12]:

public sealed class TimePlugin
{
    [KernelFunction,Description("To get current time and date")]
    [return: Description("Returns current date and time in dddd dd-MMM-yyyy hh:mm:tt")]
    public string GetToday()
    {
        return DateTime.Now.ToString("dddd dd-MMM-yyyy hh:mm:tt");
    }

    [KernelFunction,Description("Get the past Week day name based current date")]
    [return: Description("Returns date and time along with week name")]
    public string WeekDayBeforeNoOfDays([Description("The interger past no of days to")]int NoOfDaysBefore)
    {
       
        return DateTime.Now.AddDays(NoOfDaysBefore*-1).ToString("dddd dd-MMM-yyyy hh:mm:tt");
    }
 
}
kernel.Plugins.Clear();
kernel.Plugins.AddFromType<TimePlugin>();
// kernel.ImportPluginFromType<TimePlugin>();


In [13]:
IChatCompletionService chat = kernel.GetRequiredService<IChatCompletionService>();

 // Get the chat completions
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var results = await chat.GetChatMessageContentAsync("what day will come 5 days before and also include the time from now.",openAIPromptExecutionSettings,kernel);


In [14]:
Console.WriteLine(results);

The day that will come 5 days before today, including the current time, is Thursday, April 18, 2024, at 11:04 PM.
